In [1]:
import subprocess
import os
subprocess.check_output('ls')

b'Data\nLoad_Data.ipynb\nMultiClass_Classification_Practice.ipynb\nMusicClassification.ipynb\nREADME.md\ntechnology.md\n'

In [2]:
cwd_bytes = subprocess.check_output("pwd")
cwd = cwd_bytes.decode("utf-8").rstrip("\n") + "/"
cwd

'/Users/Miles/desktop/Berkeley_Camp/MusicClassification/'

In [3]:
features_3_path  = "Data/features_3_sec.csv"
features_30_path = "Data/features_30_sec.csv"
genres_path = "Data/genres_original/"
images_path = "Data/images_original/"

In [4]:
# Roadmap:
# 1: List genre directories in genres path and images path
# 2.using ls, add to list to create list of files in all genres genres and images
# 3. Then, Add each file to mongodb
byte_genres = subprocess.check_output(["ls", cwd + genres_path])
genre_folder = byte_genres.decode("utf-8").split("\n")
genre_folder.pop(-1)
genre_folder

['blues',
 'classical',
 'country',
 'disco',
 'hiphop',
 'jazz',
 'metal',
 'pop',
 'reggae',
 'rock']

In [5]:
byte_images = subprocess.check_output(["ls", cwd + images_path])
images_folder = byte_images.decode("utf-8").split("\n")
images_folder.pop(-1)
images_folder


['blues',
 'classical',
 'country',
 'disco',
 'hiphop',
 'jazz',
 'metal',
 'pop',
 'reggae',
 'rock']

In [6]:
# For both genres and images folders: list all files
# in each folder. Add to python list, iterate through this list,
# put in mongodatabase using mongofiles

for folder in genre_folder:
    byte_files = subprocess.check_output(["ls", cwd + genres_path + folder])
    files = byte_files.decode("utf-8").split("\n")
    files.pop(-1)

    for file in files:
        file_path = genres_path + folder + "/" + file
        os.system("mongofiles -d=Music_db put " + file_path)
        
        
        


In [7]:
for folder in images_folder:
    byte_files = subprocess.check_output(["ls", cwd + images_path + folder])
    files = byte_files.decode("utf-8").split("\n")
    files.pop(-1)

    for file in files:
        file_path = images_path + folder + "/" + file
        os.system("mongofiles -d=Music_db put " + file_path)

In [8]:
# Saves csv files as Mongodb collections
os.system(f"mongoimport -d Music_db -c feat_3 --type csv --file {features_3_path} --headerline")
os.system(f"mongoimport -d Music_db -c feat_30 --type csv --file {features_30_path} --headerline")

0

0